In [1]:
# импорт библиотек, необходимых для работы
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from scipy.stats import f_oneway
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Исправление датасета #

## · Отсортируем данные по дню недели и времени суток

In [ ]:
# открываем csv файл 
filename = 'sensor.csv'
data = pd.read_csv(filename) 

# конвертируем значения timestamp в datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])

# создаем пустые объекты типа Series
time_weekDays = pd.Series([])
time_dayTimes = pd.Series([])

# определяем день недели и время суток
for i in tqdm(range(data.shape[0])):
    
    # определяем день недели
    if datetime.isoweekday((data['timestamp'][i])) == 1:
        time_weekDays[i] = 'Mon'
    elif datetime.isoweekday((data['timestamp'][i])) == 2:
        time_weekDays[i] = 'Tue'
    elif datetime.isoweekday((data['timestamp'][i])) == 3:
        time_weekDays[i] = 'Wed'
    elif datetime.isoweekday((data['timestamp'][i])) == 4:
        time_weekDays[i] = 'Thu'
    elif datetime.isoweekday((data['timestamp'][i])) == 5:
        time_weekDays[i] = 'Fri'
    elif datetime.isoweekday((data['timestamp'][i])) == 6:
        time_weekDays[i] = 'Sat'
    else:
        time_weekDays[i] = 'Sun'
        
    # определяем время
    if (data['timestamp'][i].hour >= 4) and (data['timestamp'][i].hour < 12):
        time_dayTimes[i] = 'Morning'
    elif (data['timestamp'][i].hour >= 12) and (data['timestamp'][i].hour < 18):
        time_dayTimes[i] = 'Noon'
    elif (data['timestamp'][i].hour >= 18) and (data['timestamp'][i].hour < 23):
        time_dayTimes[i] = 'Evening'
    else:
        time_dayTimes[i] = 'Night'

# вставляем полученный столбец со значениями времени суток
data.insert(2, 'time_dayTimes', time_dayTimes)
# вставляем полученный столбец со значениями дней недели
data.insert(2, 'time_weekDays', time_weekDays)
# сохраняем наш измененный DataFrame в csv
data.to_csv('sensor2.csv', encoding='utf-8', index=False)

## · Посмотрим на boxplot'ы

In [ ]:
# посмотрим на обработанные данные в виде boxplot'ов

# открываем csv файл
filename = 'sensor2.csv'
data = pd.read_csv(filename)

# удаляем значения со статусами RECOVERING и BROKEN
data_normal = data.loc[~data['machine_status'].isin(['RECOVERING', 'BROKEN'])] 

for i in tqdm(range(0, 52)): # проходимся по всем сенсорам
    fig, axes = plt.subplots(nrows = 1, ncols = 2) # создаем окно для вывода графиков в обрабатываемом диапазоне (с сеткой 2 на 2)
    string = ''
    if i < 10: # так как в названиях сенсора есть 00, 01, 02... (жуть)
        string = '0' + str(i)
    else:
        string = str(i)
    
    # строим графики по значениям со статусом NORMAL
    sns.boxplot(x = data_normal['sensor_' + string], y = data_normal['time_weekDays'], ax = axes[0], color = 'lightgreen', showfliers = False)
    sns.boxplot(x = data_normal['sensor_' + string], y = data_normal['time_dayTimes'], ax = axes[1], color = 'lightgreen', showfliers = False)
    
    plt.subplots_adjust(wspace = 0.5, hspace = 0.5) # обозначим расстояние между графиками
    plt.show()

## · Проведем тест ANOVA

In [ ]:
filename = 'sensor3.csv'
data = pd.read_csv(filename)
# data = data.loc[~data['machine_status'].isin(['RECOVERING', 'BROKEN'])]

data_ = data.drop(columns = ['ID', 'timestamp', 'time_weekDays', 'time_dayTimes', 'machine_status'], axis = 1)
sensors = list(data_.columns.values)
days =  ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
for sensor in sensors:
    days_value = []
    for i in range(0, len(days)):
        days_value.append(data.loc[data['time_weekDays'].isin([days[i]]), sensor])
    test = f_oneway(days_value[0], days_value[1], days_value[2], days_value[3], days_value[4], days_value[5], days_value[6])
    print(f'Сенсор: {sensor}')
    print(f'Результат теста: {test}')
    print('\n')

## · Заполним пропуски в данных

In [ ]:
filename = 'sensor2.csv'
data = pd.read_csv(filename)

# Посмотрим % отсутствующих показаний у каждого сенсора
print('До фикса')
for col in data.columns:
    if 'sensor_' in col:
        pct_missing = np.mean(data[col].isnull())
        print('{} - {}%'.format(col, round(pct_missing * 100, 3)))
print('\n')

# у sensor_15 вообще нет данных, а у sensor_50 они заполненны лишь на 65%, поэтому их можно удалить
del data['sensor_15']
del data['sensor_50']

# отметим сенсоры с отсутствующими показаниями
# sensor_00 - 5%, sensor_06 - 2%, sensor_07 - 2%
# sensor_08 - 2%, sensor_09 - 2%, sensor_51 - 7%

# будем заполнять отсутствующие данные по их медиане в конкретный день недели 
days =  ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
sensors = ['sensor_00', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09', 'sensor_51']

for sensor in sensors: # пройдемся по всем сенсорам
    print(sensor)
    for day in days: # берем показания сенсора в каждый день недели
        print(day)
        data_com = data.loc[data['time_weekDays'].isin([day])] # оставляем показания только за интересующий нас день
        data_com = data_com.loc[ : , ['ID', sensor]] # берем показания нужного нам сенсора и ID (для индексации)
        if data_com[sensor].isnull().sum() > 0: # на всякий случай проверяем есть ли вообще незаполненные показания
            data_com[sensor] = data_com[sensor].fillna(data_com[sensor].median()) # заполняем недостающие показания медианой столбца
            for i in range(0, len(data[sensor])): # проходимся по каждому показанию нужного сенсора
                if (np.isnan(data.loc[i, sensor]) or data.loc[i, sensor] == 0) and data.loc[i, 'time_weekDays'] == day: # если значение показанию отсутствует и день недели совпадает, то
                    idx = data_com.index[data_com['ID'] == i].tolist()[0] # находим нужный индекс в сводной таблице с обработанными значениями
                    data.loc[i, sensor] = data_com.loc[idx, sensor] # записываем в основную таблицу значение из сводной по соответствующему индексу
    print('\n')      

# Посмотрим % отсутствующих показаний у каждого сенсора
print('После фикса')
for col in data.columns:
    if 'sensor_' in col:
        pct_missing = np.mean(data[col].isnull())
        print('{} - {}%'.format(col, round(pct_missing * 100, 3))) 

data.to_csv('sensor3.csv', encoding='utf-8', index=False)

## · Построим графики интервалов беспрерывной работы

In [ ]:
filename = 'sensor3.csv'
data = pd.read_csv(filename)

data_broken = data.loc[data['machine_status'].isin(['BROKEN'])] # оставляем только показания при состоянии BROKEN
broken_ids = data_broken.index.values # получаем список индексов поломок
broken_count = len(broken_ids) # получаем количество поломок

for i in range(0, broken_count - 2):
    print(f'Интервал: {i+1}')
    if i == 0:
        start = 1
    else:
        start = broken_ids[i] + 1 # задаем начальное значение среза, так как используем .iloc прибавляем единицу
    end = broken_ids[i + 1] # задаем конечное значение среза
    diapason = data.iloc[start:end, :] # выполняем срез по заданному диапазону
    fig, axes = plt.subplots(nrows = 7, ncols = 8) # создаем окно для вывода графиков в обрабатываемом диапазоне (с сеткой 7 на 8)
    # позиции для установки графика сенсора на свою позицию в сетке окна
    row = 0
    col = 0
    for i in tqdm(range(0, 52)): # проходимся по всем сенсорам
        if i not in [15, 50]:
            string = ''
            if i < 10: # так как в названиях сенсора есть 00, 01, 02... (жуть)
                string = '0' + str(i)
            else:
                string = str(i)
            # строим график по значениям сенсора
            diapason.plot(
                x = 'ID', 
                y = 'sensor_' + string, 
                ax = axes[row, col], # положение графика в сетке окна
                legend = False, # легенда мешает при выводе
                xlabel = 'sensor_' + string # чтобы понятно было где чей график
            )
            col += 1 # после установки графика смещаем следующий в следующую колонну
            if col == 8: # сетка 7 на 8, поэтому, когда колонны в строке кончаются, переходим на следующую строку
                row += 1 # переход на следующую строку
                col = 0 # новая строка - нумерация колонн сначала
    plt.subplots_adjust(wspace = 0.75, hspace = 0.75) # обозначаем расстояние между графиками
    plt.show()

## · Построим тепловые карты для оценки корреляции

In [ ]:
filename = 'sensor3.csv'
data = pd.read_csv(filename)
new_data = data.loc[ : , ['timestamp', 'sensor_14', 'sensor_19',  'sensor_20', 'sensor_21', 'sensor_22',
                           'sensor_24', 'sensor_25', 'sensor_26', 'sensor_28', 'sensor_31', 'sensor_33',
                           'sensor_34', 'sensor_41', 'sensor_42', 'sensor_49', 'sensor_51']]
sns.heatmap(data.corr(), square = True)
plt.show()
sns.heatmap(new_data.corr(), annot = True, square = True)
plt.show()